In [1]:
import os
import pydicom
import nibabel as nib
import numpy as np
from tqdm import tqdm  # 진행 상태 표시

In [2]:
def find_dicom_folders(base_path, target_folder_names):
    """
    특정 이름을 가진 폴더들을 재귀적으로 탐색해 DICOM 파일이 있는 폴더와 조건 불만족 폴더를 반환.

    Parameters:
        base_path (str): 탐색할 최상위 폴더 경로.
        target_folder_names (list): 찾고자 하는 폴더 이름들의 리스트.

    Returns:
        tuple:
            - dicom_folders (list): DICOM 파일이 있는 폴더 경로 리스트.
            - failed_folders (list): DICOM 파일이 없거나 조건을 만족하지 않는 폴더 경로 리스트.
    """
    dicom_folders = []
    failed_folders = []

    def recursive_search(current_path):
        """
        타겟 폴더를 찾은 후 하위 폴더까지 재귀적으로 탐색.
        """
        for root, dirs, files in os.walk(current_path):
            for target_folder_name in target_folder_names:
                # 타겟 폴더 탐색
                if target_folder_name in dirs:
                    target_path = os.path.join(root, target_folder_name)
                    print(f"🔍 [타겟 폴더 발견]: {target_path}")

                    # 타겟 폴더 내부 탐색 (여러 시점 폴더 포함)
                    for sub_root, sub_dirs, sub_files in os.walk(target_path):
                        for sub_dir in sub_dirs:
                            sub_dir_path = os.path.join(sub_root, sub_dir)
                            dcm_files = [f for f in os.listdir(sub_dir_path) if f.lower().endswith('.dcm')]

                            if dcm_files:  # DICOM 파일 존재
                                dicom_folders.append(sub_dir_path)
                                print(f"✅ [DICOM 발견]: {sub_dir_path}")
                            else:
                                failed_folders.append(sub_dir_path)
                                print(f"❌ [DICOM 없음]: {sub_dir_path}")

    # 최상위 폴더부터 시작
    recursive_search(base_path)

    return dicom_folders, failed_folders


def dicom_to_nifti(dicom_folder, output_path):
    """
    DICOM 파일들이 있는 폴더를 NIfTI로 변환하는 함수.

    Parameters:
        dicom_folder (str): DICOM 파일들이 있는 폴더 경로.
        output_path (str): 변환된 NIfTI 파일 저장 경로 (.nii.gz).
    """
    # DICOM 파일 읽기
    dicom_files = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.lower().endswith('.dcm')]
    dicom_files.sort()  # 파일 정렬

    if not dicom_files:
        print(f"❌ [오류] {dicom_folder}에 DICOM 파일이 없습니다.")
        return
    
    try:
        # 3D 볼륨 생성
        slices = [pydicom.dcmread(dcm).pixel_array for dcm in dicom_files]
        volume = np.stack(slices, axis=-1)  # HxWxD 형태로 스택

        # NIfTI 파일 저장
        affine = np.eye(4)  # 기본 affine 행렬
        nifti_img = nib.Nifti1Image(volume, affine)
        nib.save(nifti_img, output_path)
        print(f"✅ [NIfTI 저장 완료]: {output_path}")
    except Exception as e:
        print(f"❌ [오류 발생]: {dicom_folder} - {e}")


def convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path):
    """
    ADNI 폴더 내 특정 이름을 가진 여러 폴더의 DICOM 파일을 NIfTI로 변환하는 함수.

    Parameters:
        base_path (str): 최상위 폴더 경로.
        target_folder_names (list): 찾고자 하는 폴더 이름들의 리스트.
        output_base_path (str): 변환된 NIfTI 파일 저장할 루트 경로.
    """
    # Step 1: 대상 DICOM 폴더 찾기
    print("🔎 DICOM 폴더를 탐색 중입니다...")
    dicom_folders, failed_folders = find_dicom_folders(base_path, target_folder_names)
    print(f"✅ 총 {len(dicom_folders)}개의 DICOM 폴더를 찾았습니다.")
    print(f"❌ 총 {len(failed_folders)}개의 폴더에서 DICOM 파일을 찾을 수 없습니다.")

    # Step 2: 각 DICOM 폴더를 NIfTI로 변환
    for dicom_folder in tqdm(dicom_folders, desc="DICOM → NIfTI 변환 진행"):
        # 출력 경로 설정 (환자ID/날짜 구조 유지)
        relative_path = os.path.relpath(dicom_folder, base_path)
        output_folder = os.path.join(output_base_path, os.path.dirname(relative_path))
        os.makedirs(output_folder, exist_ok=True)
        
        output_file = os.path.join(output_folder, f"{os.path.basename(dicom_folder)}.nii.gz")
        dicom_to_nifti(dicom_folder, output_file)

    # 실패한 폴더 출력
    if failed_folders:
        print("\n❌ [조건을 만족하지 못한 폴더]:")
        for folder in failed_folders:
            print(f" - {folder}")

In [3]:
 # 최상위 ADNI 폴더 경로 설정
base_path = "E:\\MRI 파일\\ADNI4"

# 찾고자 하는 타겟 폴더 이름 리스트
target_folder_names = [
"Accelerated_Sagittal_MPRAGE",
"Sagittal_3D_Accelerated_MPRAGE",
"Accelerated_Sag_IR-FSPGR",
"Accelerated_Sagittal_IR-FSPGR",
"Sag_Accel_IR-FSPGR",
"Sagittal_3D_Accelerated_0_angle_MPRAGE",
"Accelerated_Sagittal_MPRAGE_Phase_A-P",
"Accelerated_Sagittal_MPRAGE__MSV21",
"Accelerated_Sagittal_MPRAGE__MSV21_",
"Accelerated_Sagittal_MPRAGE__MSV21_RPT",
"Accelerated_Sagittal_IR-FSPGR__MSV21",
"Accelerated_Sagittal_IR-FSPGR__MSV21_",
"Accelerated_Sagittal_IR-FSPGR__MSV21__RPT",
"Accelerated_Sagittal_IR-FSPGR__MSV22",
"Accelerated_Sagittal_IR-FSPGR__MSV22_",
"Sagittal_3D_Accelerated_MPRAGE_MSV21_"
]

# NIfTI 파일 저장 경로
output_base_path = "./niis"

# 변환 함수 호출
convert_adni_dicom_to_nifti(base_path, target_folder_names, output_base_path)

🔎 DICOM 폴더를 탐색 중입니다...
🔍 [타겟 폴더 발견]: E:\MRI 파일\ADNI4\003_S_10042\2024-02-13\Accelerated_Sagittal_MPRAGE__MSV21_
✅ [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10042\2024-02-13\Accelerated_Sagittal_MPRAGE__MSV21_\I10365678
🔍 [타겟 폴더 발견]: E:\MRI 파일\ADNI4\003_S_10073\2024-04-24\Accelerated_Sagittal_MPRAGE__MSV21_
✅ [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10073\2024-04-24\Accelerated_Sagittal_MPRAGE__MSV21_\I10454329
🔍 [타겟 폴더 발견]: E:\MRI 파일\ADNI4\003_S_10082\2024-04-30\Accelerated_Sagittal_MPRAGE__MSV21_
✅ [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10082\2024-04-30\Accelerated_Sagittal_MPRAGE__MSV21_\I10454163
🔍 [타겟 폴더 발견]: E:\MRI 파일\ADNI4\003_S_10158\2024-07-11\Accelerated_Sagittal_MPRAGE__MSV21_
✅ [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_10158\2024-07-11\Accelerated_Sagittal_MPRAGE__MSV21_\I10883407
🔍 [타겟 폴더 발견]: E:\MRI 파일\ADNI4\003_S_4441\2024-04-23\Accelerated_Sagittal_MPRAGE__MSV21_
✅ [DICOM 발견]: E:\MRI 파일\ADNI4\003_S_4441\2024-04-23\Accelerated_Sagittal_MPRAGE__MSV21_\I10454353
🔍 [타겟 폴더 발견]: E:\MRI 파일\ADNI4\003_S_490

DICOM → NIfTI 변환 진행:   0%|          | 1/237 [00:00<03:30,  1.12it/s]

✅ [NIfTI 저장 완료]: ./niis\003_S_10042\2024-02-13\Accelerated_Sagittal_MPRAGE__MSV21_\I10365678.nii.gz


DICOM → NIfTI 변환 진행:   1%|          | 2/237 [00:01<03:32,  1.11it/s]

✅ [NIfTI 저장 완료]: ./niis\003_S_10073\2024-04-24\Accelerated_Sagittal_MPRAGE__MSV21_\I10454329.nii.gz


DICOM → NIfTI 변환 진행:   1%|          | 2/237 [00:02<04:00,  1.02s/it]


KeyboardInterrupt: 